In [1]:
import pandas as pd 
data = pd.read_csv('user_journey_raw.csv')
print(data.head())

   user_id  session_id subscription_type  \
0     1516     2980231            Annual   
1     1516     2980248            Annual   
2     1516     2992252            Annual   
3     1516     3070491            Annual   
4     1516     3709807            Annual   

                                        user_journey  
0  Homepage-Log in-Log in-Log in-Log in-Log in-Lo...  
1  Other-Sign up-Sign up-Sign up-Sign up-Sign up-...  
2          Log in-Log in-Log in-Log in-Log in-Log in  
3  Homepage-Log in-Log in-Log in-Log in-Log in-Lo...  
4  Log in-Log in-Log in-Log in-Log in-Log in-Log ...  


In [2]:
print(data.shape)

(9935, 4)


In [3]:
print(data.isnull().mean() * 100) #percentage of the missing values

user_id              0.0
session_id           0.0
subscription_type    0.0
user_journey         0.0
dtype: float64


In [4]:
print(data.dtypes)

user_id               int64
session_id            int64
subscription_type    object
user_journey         object
dtype: object


In [5]:
print(data['user_journey'].unique()) #detect inconsistent data types

['Homepage-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Log in-Other'
 'Other-Sign up-Sign up-Sign up-Sign up-Sign up-Log in-Log in-Log in-Log in-Log in-Log in'
 'Log in-Log in-Log in-Log in-Log in-Log in' ...
 'Other-Career tracks-Career tracks-Courses-Courses-Courses-Courses-Career tracks-Career tracks-Sign up-Sign up-Sign up-Sign up-Sign up-Sign up-Sign up-Sign up-Sign up'
 'Other-Other-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon-Coupon'
 'Homepage-Upcoming courses']


In [6]:
def remove_page_duplicates(data, target_column='user_journey'):
    """
    Removes sequences of repeating pages in the user journey strings.
    
    Parameters:
    - data: DataFrame containing the data.
    - target_column: Column name containing user journey strings (default 'user_journey').
    
    Returns:
    - A new DataFrame with cleaned-up user journey strings.
    """
    # Copy the original dataframe to avoid modifying it
    cleaned_data = data.copy()
    
    # Function to remove consecutive duplicates from a journey string
    def clean_journey(journey):
        pages = journey.split('-')  # Split into individual pages
        cleaned_pages = [pages[0]]  # Initialize with the first page
        for page in pages[1:]:
            if page != cleaned_pages[-1]:  # Add only if different from the previous
                cleaned_pages.append(page)
        return '-'.join(cleaned_pages)  # Rejoin into a string
    # Apply the cleaning function to the target column
    cleaned_data[target_column] = cleaned_data[target_column].apply(clean_journey)
    
    return cleaned_data

In [8]:
# Call the function
cleaned_data = remove_page_duplicates(data)

# Output DataFrame
print(cleaned_data)

      user_id  session_id subscription_type           user_journey
0        1516     2980231            Annual  Homepage-Log in-Other
1        1516     2980248            Annual   Other-Sign up-Log in
2        1516     2992252            Annual                 Log in
3        1516     3070491            Annual        Homepage-Log in
4        1516     3709807            Annual                 Log in
...       ...         ...               ...                    ...
9930   509095     4487613            Annual                  Other
9931   509095     4842565            Annual                  Other
9932   509095     4843103            Annual                  Other
9933   509095     4845316            Annual                  Other
9934   509096     4845427            Annual           Other-Coupon

[9935 rows x 4 columns]


In [15]:
def group_by(grouped_df, group_column='user_id', target_column='user_journey', sessions='All', count_from='last'):
    """
    Groups user sessions into a single journey string for each user.

    Parameters:
    - data: DataFrame containing the data.
    - group_column: Column name to group by (default 'user_id').
    - target_column: Column containing journey strings to combine (default 'user_journey').
    - sessions: Number of sessions to group ('All' or an integer, default 'All').
    - count_from: 'first' or 'last', where to start grouping if sessions is an integer (default 'last').

    Returns:
    - A new DataFrame with grouped journey strings.
    """
    # Copy the original dataframe to avoid modification
    grouped_df = cleaned_data.copy()

    # Function to group journeys
    def group_journeys(sub_df):
        if sessions == 'All':
            return '-'.join(sub_df[target_column])
        elif count_from == 'first':
            return '-'.join(sub_df[target_column].iloc[:sessions])
        elif count_from == 'last':
            return '-'.join(sub_df[target_column].iloc[-sessions:])
        else:
            raise ValueError("Invalid value for 'count_from'. Choose 'first' or 'last'.")

    # Perform grouping and apply function
    '''
    group_keys=False: Ensures that only the target DataFrame rows are passed to the apply function, excluding the grouping keys.
    '''
    result = (
        grouped_df.groupby(group_column, group_keys=False)
        .apply(group_journeys)
        .reset_index(name=target_column)
    )
    
    return result

In [17]:
cleaned_grouped_df = group_by(cleaned_data)
print(cleaned_grouped_df)

      user_id                                       user_journey
0        1516  Homepage-Log in-Other-Other-Sign up-Log in-Log...
1        3395  Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2       10107  Homepage-Homepage-Career tracks-Homepage-Caree...
3       11145  Homepage-Log in-Homepage-Log in-Homepage-Log i...
4       12400  Homepage-Career tracks-Sign up-Log in-Other-Ca...
...       ...                                                ...
1345   509060                                        Other-Other
1346   509061                                             Coupon
1347   509085                                             Coupon
1348   509095                            Other-Other-Other-Other
1349   509096                                       Other-Coupon

[1350 rows x 2 columns]


C:\Users\21693\AppData\Local\Temp\ipykernel_6700\4269526294.py:35: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(group_journeys)


In [18]:
print(cleaned_grouped_df.head())

   user_id                                       user_journey
0     1516  Homepage-Log in-Other-Other-Sign up-Log in-Log...
1     3395  Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2    10107  Homepage-Homepage-Career tracks-Homepage-Caree...
3    11145  Homepage-Log in-Homepage-Log in-Homepage-Log i...
4    12400  Homepage-Career tracks-Sign up-Log in-Other-Ca...


In [19]:
def remove_pages(dataframe , pages, target_column='user_journey'):
    """
    Removes specified pages from the user journey strings in the DataFrame.
    
    Parameters:
    - data: DataFrame containing the data.
    - pages: List of page strings to remove.
    - target_column: Column containing user journey strings (default 'user_journey').
    
    Returns:
    - A new DataFrame with the specified pages removed.
    """
    # Copy the original dataframe to avoid modifying it
    dataframe = cleaned_grouped_df.copy()
    
    # Function to remove pages from a journey string
    def clean_journey(journey):
        journey_pages = journey.split('-')  # Split into individual pages
        filtered_pages = [page for page in journey_pages if page not in pages]  # Filter out unwanted pages
        return '-'.join(filtered_pages)  # Rejoin into a string
    
    # Apply the cleaning function to the target column
    dataframe[target_column] = dataframe[target_column].apply(clean_journey)
    
    return dataframe

In [22]:
# Define the pages to remove
pages_to_remove = ['Log in', 'Log out']

# Apply the remove_pages function
processed_dataframe = remove_pages(cleaned_grouped_df, pages_to_remove)

# Display the result
print(processed_dataframe)

      user_id                                       user_journey
0        1516  Homepage-Other-Other-Sign up-Homepage-Checkout...
1        3395  Other-Pricing-Sign up-Homepage-Pricing-Pricing...
2       10107  Homepage-Homepage-Career tracks-Homepage-Caree...
3       11145  Homepage-Homepage-Homepage-Homepage-Homepage-H...
4       12400  Homepage-Career tracks-Sign up-Other-Career tr...
...       ...                                                ...
1345   509060                                        Other-Other
1346   509061                                             Coupon
1347   509085                                             Coupon
1348   509095                            Other-Other-Other-Other
1349   509096                                       Other-Coupon

[1350 rows x 2 columns]


In [23]:
# Call the function
final_data = remove_page_duplicates(processed_dataframe)

# Output DataFrame
print(final_data)

      user_id                                       user_journey
0        1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...
1        3395    Other-Pricing-Sign up-Homepage-Pricing-Checkout
2       10107  Homepage-Career tracks-Homepage-Career tracks-...
3       11145                                  Homepage-Checkout
4       12400  Homepage-Career tracks-Sign up-Other-Career tr...
...       ...                                                ...
1345   509060                                              Other
1346   509061                                             Coupon
1347   509085                                             Coupon
1348   509095                                              Other
1349   509096                                       Other-Coupon

[1350 rows x 2 columns]


In [24]:
print(data.head())

   user_id  session_id subscription_type  \
0     1516     2980231            Annual   
1     1516     2980248            Annual   
2     1516     2992252            Annual   
3     1516     3070491            Annual   
4     1516     3709807            Annual   

                                        user_journey  
0  Homepage-Log in-Log in-Log in-Log in-Log in-Lo...  
1  Other-Sign up-Sign up-Sign up-Sign up-Sign up-...  
2          Log in-Log in-Log in-Log in-Log in-Log in  
3  Homepage-Log in-Log in-Log in-Log in-Log in-Lo...  
4  Log in-Log in-Log in-Log in-Log in-Log in-Log ...  


In [25]:
final_data = pd.merge(
    final_data,  # The processed DataFrame
    data[['user_id', 'subscription_type']],  # Selecting only user_id and subscription_type from the original DataFrame
    on='user_id',  # Merge on the user_id column
    how='left'  # Use 'left' join to preserve all rows in final_data
)

In [26]:
print(final_data.head())

   user_id                                       user_journey  \
0     1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...   
1     1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...   
2     1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...   
3     1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...   
4     1516  Homepage-Other-Sign up-Homepage-Checkout-Coupo...   

  subscription_type  
0            Annual  
1            Annual  
2            Annual  
3            Annual  
4            Annual  


In [27]:
# Export final_data to a CSV file
final_data.to_csv('final_data.csv', index=False)